In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random

import scipy.special as scs
from scipy import optimize

In [21]:
def nMGA(exp,N):
    # Input parameters ####################
    #T_max = 50
    n_E = exp      # int; initial infected population
    n_I = 0
    n_R = 0        # int; recovered population

    T = T_max      # float; maximum elapsed time
    _t = 0.0       # float; start time


    V = N          # float; spatial parameter
    
    _beta = 2.4
    _gamma_E = 2.5   # float; rate of infection
    _gamma_R = 1.5   # float; rate of recovery

    
    t_SE = 0.0     # float; initialising times for processes
    t_EI = 0.0
    t_IR = 0.0
    
    t_el = np.array([t_SE, t_EI, t_IR])
    alpha_i = np.array([1,1,1])
    kappa_i = np.array([2,4,19])
    
    
    
    n_S = N - n_I - n_E  # Computing susceptible population


    # Initialize results list ###############

    S = []
    E = []
    I = []
    R = []
    t = []

    S.append(n_S)
    E.append(n_E)
    I.append(n_I)
    R.append(n_R)
    t.append(_t)

    
    def psi(tau, alpha, kappa):
        # Function that defines the probability distribution
        #ans = alpha*np.exp(-alpha*tau)
        if(alpha<0):
            alpha=1
        ans = tau**(alpha-1)*np.exp(-tau/kappa)/(scs.gamma(alpha)*kappa*alpha)
        return ans
    
    
    def Psi(tau, alpha, kappa):
        # Function defining survival probability
        #ans = np.exp(-alpha*tau)
        if(alpha==0):
            alpha=1
        ans = scs.gammaincc(alpha,tau/kappa)
        print("Alpha "+str(alpha))
        print("Kappa "+str(kappa))
        print("Tau   "+str(tau))
        print(ans)
        return ans
    
    
    def solvePhi(deltaT,u):
        prod=1
        for i in range(0,len(t_el)):
            prod = prod*Psi(t_el[i]+deltaT,alpha_i[i],kappa_i[i])/Psi(t_el[i],alpha_i[i],kappa_i[i])
        return prod - u
    
    # Loop to compute results at every time-step

    while _t < T:                                        
        if n_I + n_E == 0:   # <+++ CHECK THIS!       
            break                                     # Stop the loop if there are no infected individuals

        
        alpha_i[0] = _beta * n_S * n_I / V
        alpha_i[1] = _gamma_E * n_E
        alpha_i[2] = _gamma_R * n_I
        
        print("A0 "+str(alpha_i[0])+"A1 "+str(alpha_i[1])+"A2 "+str(alpha_i[2]))
        
        u = random.uniform(0.0, 1.0)
    
        dt = optimize.newton(solvePhi,0.0,args=(u,),tol=1E-10)
        _t = _t + dt
        
        l = np.array([0.0, 0.0, 0.0])
        

        for i in range(0,len(t_el)):
            l[i] = psi(t_el[i]+dt,alpha_i[i],kappa_i[i])/Psi(t_el[i]+dt,alpha_i[i],kappa_i[i])
        
        sum_li = np.sum(l)
        
        l = l/sum_li
        
        r = random.uniform(0.0, 1.0)
        
        if r < l[0]:
            n_S = n_S - 1
            n_E = n_E + 1
            t_el[0]=0
            t_el = t_el + [0,dt,dt]
        elif r < l[0]+l[1]:
            n_E = n_E - 1
            n_I = n_I + 1
            t_el[1]=0
            t_el = t_el + [dt,0,dt]
        else:
            n_I = n_I - 1
            n_R = n_R + 1
            t_el[2]=0
            t_el = t_el + [dt,dt,0]
        
        S.append(n_S)
        E.append(n_E)
        I.append(n_I)
        R.append(n_R)
        t.append(_t)

#     plt.plot(t,S)
#     plt.plot(t,I)
#     plt.plot(t,R)
    return t,np.array(S)/N,np.array(E)/N,np.array(I)/N,np.array(R)/N

    #########################################

In [ ]:
total_iter = 10

T_max = 40
t_av = np.linspace(0.00000001,T_max,10*T_max)
S_av = np.zeros(len(t_av),float)
E_av = np.zeros(len(t_av),float)
I_av = np.zeros(len(t_av),float)
R_av = np.zeros(len(t_av),float)
#print(t_av)
N_tot = 10000
exp_0 = 100

len_t = []

for i in range(0,total_iter):
    t,S,E,I,R = nMGA(exp_0,N_tot)
    len_t.append(len(t))
    k=0
    j_max = 0
    s_at_k = 0
    e_at_k = 0
    inf_at_k = 0
    r_at_k = 0
    while(k<len(t_av)):
        counter = 0
        s_at_k = 0
        e_at_k = 0
        inf_at_k = 0
        r_at_k = 0
        for j in range(j_max,len(t)):
            if(t[j]<t_av[k]):
                inf_at_k = inf_at_k+I[j]
                s_at_k = s_at_k + S[j]
                r_at_k = r_at_k + R[j]
                e_at_k = e_at_k + E[j]
                counter += 1
            else:
                j_max = j+1
                if(counter !=0):
                    I_av[k] = I_av[k]+inf_at_k/counter
                    S_av[k] = S_av[k]+s_at_k/counter
                    E_av[k] = E_av[k]+e_at_k/counter
                    R_av[k] = R_av[k]+r_at_k/counter
                break
        k=k+1
    
    
# t_ex, S_ex, E_ex, I_ex, R_ex = SEIR(exp_0,N_tot)

# plt.plot(t_ex,I_ex, linewidth=3, color='red')
plt.plot(t_av,S_av/total_iter)
plt.plot(t_av,E_av/total_iter)
plt.plot(t_av,I_av/total_iter)
plt.plot(t_av,R_av/total_iter)

plt.xlabel('Time')
plt.ylabel('Fraction of population')
#plt.savefig('../images/Gillespie_100avg_i0_100_3.png')
plt.show()

In [ ]:
plt.plot(t_av,S_av/total_iter)
plt.plot(t_av,E_av/total_iter)
plt.plot(t_av,I_av/total_iter)
plt.plot(t_av,R_av/total_iter)

plt.xlabel('Time')
plt.ylabel('Fraction of population')
#plt.savefig('../images/Gillespie_100avg_i0_100_3.png')
plt.show()

In [22]:
T_max = 40
t,s,e,i,r = nMGA(100,1000)

A0 0A1 250A2 0
Alpha 1
Kappa 2
Tau   0.0
1.0
Alpha 1
Kappa 2
Tau   0.0
1.0
Alpha 250
Kappa 4
Tau   0.0
1.0
Alpha 250
Kappa 4
Tau   0.0
1.0
Alpha 1
Kappa 19
Tau   0.0
1.0
Alpha 1
Kappa 19
Tau   0.0
1.0
Alpha 1
Kappa 2
Tau   0.0001
0.9999500012499791
Alpha 1
Kappa 2
Tau   0.0
1.0
Alpha 250
Kappa 4
Tau   0.0001
1.0
Alpha 250
Kappa 4
Tau   0.0
1.0
Alpha 1
Kappa 19
Tau   0.0001
0.9999947368559556
Alpha 1
Kappa 19
Tau   0.0
1.0
Alpha 1
Kappa 2
Tau   1.27989559347677
0.5273199511459046
Alpha 1
Kappa 2
Tau   0.0
1.0
Alpha 250
Kappa 4
Tau   1.27989559347677
1.0
Alpha 250
Kappa 4
Tau   0.0
1.0
Alpha 1
Kappa 19
Tau   1.27989559347677
0.9348558562911801
Alpha 1
Kappa 19
Tau   0.0
1.0
Alpha 1
Kappa 2
Tau   1.7854082606031543
0.4095467861586555
Alpha 1
Kappa 2
Tau   0.0
1.0
Alpha 250
Kappa 4
Tau   1.7854082606031543
1.0
Alpha 250
Kappa 4
Tau   0.0
1.0
Alpha 1
Kappa 19
Tau   1.7854082606031543
0.9103111126913292
Alpha 1
Kappa 19
Tau   0.0
1.0
Alpha 1
Kappa 2
Tau   2.1224338161551444
0.346034461879108

/home/philip/.local/lib/python3.6/site-packages/ipykernel_launcher.py:52: RuntimeWarning: invalid value encountered in double_scalars


RuntimeError: Failed to converge after 50 iterations, value is nan.

In [23]:
plt.plot(t,s)
plt.plot(t,e)
plt.plot(t,i)
plt.plot(t,r)

NameError: name 't' is not defined

In [18]:
scs.gammaincc(100,0)

1.0